In [2]:
import numpy as np
import pandas as pd
import os

# Directory paths
output_dir = "C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/results"
cpi_weights_dir = "C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/cpi_weights"
price_volatility_path = "C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/price_data/II_PI_volatility.csv"

# Load the price volatility
price_volatility = pd.read_csv(price_volatility_path, index_col=0)

# Function to process individual impacts for a given year
def calculate_individual_impacts_for_year(year, unweighted_impacts_path, cpi_weights_path, output_dir):
    print(f"Processing individual impacts for {year}...")
    # Load the unweighted impacts and CPI weights
    unweighted_impacts = pd.read_csv(unweighted_impacts_path, index_col=0)
    cpi_weights = pd.read_csv(cpi_weights_path, index_col=0)
    cpi_weights.index = cpi_weights.index.str.replace('^op_', '', regex=True)

    # Process impacts
    impacts = []
    for exogenous_sector in unweighted_impacts.index:  # Use rows of unweighted impacts
        if exogenous_sector.startswith('ip_'):
            exogenous_sector = exogenous_sector[3:]

        try:
            # Extract price shock and CPI weight for the sector
            price_shock = price_volatility.loc[exogenous_sector, 'price_volatility']
            country_code, sector_code = exogenous_sector.split('_', 1)
            cpi_weight = cpi_weights.loc[exogenous_sector, country_code]

            # Calculate direct impact
            direct_impact = cpi_weight * price_shock

            # Calculate indirect impacts
            propagated_shocks = unweighted_impacts.loc[exogenous_sector]  # Get the row for the exogenous sector
            indirect_impact = (
                (cpi_weights.loc[propagated_shocks.index, country_code] * propagated_shocks)
                .drop(index=exogenous_sector, errors='ignore')  # Exclude the exogenous sector from the calculation
                .sum()
            )
        except KeyError as e:
            print(f"Error during impact calculation for {exogenous_sector}: {e}")
            continue

        # Calculate total impact
        total_impact = direct_impact + indirect_impact
        impacts.append({
            'Country': country_code,
            'Sector': sector_code,
            'Direct Impact': direct_impact,
            'Indirect Impact': indirect_impact,
            'Total Impact': total_impact
        })

    # Save results
    impacts_df = pd.DataFrame(impacts)
    output_file = os.path.join(output_dir, f"individual_impacts_{year}.csv")
    impacts_df.to_csv(output_file, index=False)
    print(f"Individual impacts for {year} saved to {output_file}")

# Function to process EU28 impacts for a given year
def calculate_eu28_impacts_for_year(year, unweighted_impacts_path, cpi_weights_path, output_dir):
    print(f"Processing EU28 impacts for {year}...")
    # Load the unweighted impacts and CPI weights
    unweighted_impacts = pd.read_csv(unweighted_impacts_path, index_col=0)
    cpi_weights = pd.read_csv(cpi_weights_path, index_col=0)
    cpi_weights.index = cpi_weights.index.str.replace('^op_', '', regex=True)

    # Process impacts
    impacts = []
    for exogenous_sector in unweighted_impacts.index:  # Use rows of unweighted impacts
        if exogenous_sector.startswith('ip_'):
            exogenous_sector = exogenous_sector[3:]

        try:
            # Extract price shock and CPI weight for the sector
            price_shock = price_volatility.loc[exogenous_sector, 'price_volatility']
            cpi_weight = cpi_weights.loc[exogenous_sector, 'EU28']

            # Calculate direct impact
            direct_impact = cpi_weight * price_shock

            # Calculate indirect impacts
            propagated_shocks = unweighted_impacts.loc[exogenous_sector]  # Get the row for the exogenous sector
            indirect_impact = (
                (cpi_weights.loc[propagated_shocks.index, 'EU28'] * propagated_shocks)
                .drop(index=exogenous_sector, errors='ignore')  # Exclude the exogenous sector
                .sum()
            )
        except KeyError as e:
            print(f"Error during impact calculation for {exogenous_sector}: {e}")
            continue

        # Calculate total impact
        total_impact = direct_impact + indirect_impact
        impacts.append({
            'Sector': exogenous_sector,
            'Direct Impact': direct_impact,
            'Indirect Impact': indirect_impact,
            'Total Impact': total_impact
        })

    # Save results
    impacts_df = pd.DataFrame(impacts)
    output_file = os.path.join(output_dir, f"eu28_impacts_{year}.csv")
    impacts_df.to_csv(output_file, index=False)
    print(f"EU28 impacts for {year} saved to {output_file}")

# Main process for all years
for year in range(2010, 2023):  # Adjust range based on available data
    unweighted_impacts_path = os.path.join(output_dir, f"unweighted_shock_impacts_{year}.csv")
    individual_weights_path = os.path.join(cpi_weights_dir, f"individual_cpi_weights_{year}.csv")
    eu28_weights_path = os.path.join(cpi_weights_dir, f"eu28_and_row_cpi_weights_{year}.csv")

    # Calculate individual and EU28 impacts for the current year
    calculate_individual_impacts_for_year(year, unweighted_impacts_path, individual_weights_path, output_dir)
    calculate_eu28_impacts_for_year(year, unweighted_impacts_path, eu28_weights_path, output_dir)

Processing individual impacts for 2010...
Individual impacts for 2010 saved to C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/results\individual_impacts_2010.csv
Processing EU28 impacts for 2010...
EU28 impacts for 2010 saved to C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/results\eu28_impacts_2010.csv
Processing individual impacts for 2011...
Individual impacts for 2011 saved to C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/results\individual_impacts_2011.csv
Processing EU28 impacts for 2011...
EU28 impacts for 2011 saved to C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/results\eu28_impacts_2011.csv
Processing individual impacts for 2012...
Individual impacts for 2012 saved to C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/results\individual_impacts_2012.csv
Processing EU28 impacts for 2012...
EU28 impacts for 2012 saved to C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/results\eu28_impacts_2012.csv
Proc